In [1]:
from vmi_analysis import coincidence_v4
from vmi_analysis import calibrations
import numpy as np
import matplotlib.pyplot as plt
file=r"J:\ctgroup\Edward\DATA\VMI\20250307\po4w30deg\30.cv4"

x,y,t,etof,itof=coincidence_v4.load_file(file)
xn,yn,tn,etofn,_=coincidence_v4.load_file(file,coincidence=False)
etof+=0.26*np.random.random_sample(len(etof))
etofn+=0.26*np.random.random_sample(len(etofn))

Loading J:\ctgroup\Edward\DATA\VMI\20250307\po4w30deg\30.mat
Loading J:\ctgroup\Edward\DATA\VMI\20250307\po4w30deg\30.cv4 (No Coincidence: 6552840 clusters, 4310489 etofs, 2948033 itofs


In [ ]:
px,py,pz=calibrations.calibration_20250303(x,y,etof,symmetrize=False)
pxn,pyn,pzn=calibrations.calibration_20250303(xn,yn,etofn,symmetrize=False)

In [ ]:
%matplotlib nbagg
import skimage

y_mask=(py>-0.1) & (py<0.1)

plt.figure(figsize=(10,20))
plt.subplot(8,1,1)

plt.hist(itof,bins=3000,range=(0,20000),color='g',alpha=0.7,label='i-ToF')

itof_edges=(6000,9000, 11580,13930,16000)
itof_masks=[(itof>itof_edges[i]) & (itof<itof_edges[i+1]) & y_mask for i in range(len(itof_edges)-1)]

pr,ptheta,pphi=np.sqrt(px**2+py**2+pz**2),np.arctan2(pz,px),np.arctan2(py,np.sqrt(px**2+pz**2))
prn,pthetan,pphin=np.sqrt(pxn**2+pyn**2+pzn**2),np.arctan2(pzn,pxn),np.arctan2(pyn,np.sqrt(pxn**2+pzn**2))
for i,mask in enumerate(itof_masks):
    plt.subplot(8,5,5+i+1)
    hist,xe,ze=np.histogram2d(px[mask],pz[mask],bins=256,range=((-0.6,0.6),(-0.6,0.6)),density=True)
    # dog=skimage.filters.difference_of_gaussians(hist,5,30)
    dog=hist
    plt.imshow(dog.T,origin='lower',aspect='equal')
    plt.axis('off')

plt.subplot(8,1,3)

yn_mask=(pyn>-0.1) & (pyn<0.1)
m=1
# hist,thetha_e,r_e=np.histogram2d((pthetan[yn_mask]+np.pi),prn[yn_mask],bins=512,range=((0,2*np.pi),(0.1,0.5)),density=True)
hist,thetha_e,r_e=np.histogram2d(ptheta[itof_masks[m]]+np.pi,pr[itof_masks[m]],bins=512,range=((0,2*np.pi),(0.1,0.5)),density=True)
hist=skimage.filters.difference_of_gaussians(hist,7,30,mode='wrap')

# ridges= skimage.filters.meijering(hist)
plt.imshow(hist.T,origin='lower',aspect='auto',extent=(-np.pi,np.pi,.1,0.5), interpolation='bilinear')


plt.subplot(2,1,2)
rr,thth=np.meshgrid(r_e[:-1],thetha_e[:-1])
xx,yy=rr*np.cos(thth),rr*np.sin(thth)
plt.contourf(xx,yy,hist,levels=100)
plt.axis('off')
plt.axis('equal')
plt.tight_layout()
# plt.contourf(,r_e[:-1]*np.sin(thetha_e)[:-1],hist,levels=100)
plt.show()